In [2]:
import os
import shutil
import time
import pandas as pd
from tqdm import tqdm      #tqdm(filelist, total = len(file_list), position=0, leave=True)
import re
from pathlib import Path
import traceback
from os.path import join
import file_function as ff
from datetime import datetime
from upload_v2 import dict_refer

In [ ]:
path = r'C:\Users\SL\Desktop\test'
ff.moveFilesToRootFolderAndRmDirs(path)

### 파일명 작성 지침 변경에 따른 부산 수정

##### 파일 전체 옮기기

In [ ]:
path = r'D:\부산 정리_v1'
dst_root = r'D:\부산 정리_v2'
ff.copy_all(path, dst_root)

##### '보증인', 집행권원, 강제집행 등 문서종류 및 보증인 수정

In [3]:
path = r'\\192.168.0.75\스캔파일\새 스캔파일(업로드)'
# filelist = os.walk(path)
file_list = [f.name for f in os.scandir(path) if f.is_file()]

for f in tqdm(file_list) :
    if f != "Thumbs.db" :
        new_name = f

        if re.search("판결문|지급명령|이행권고|화해권고", new_name) :
            new_name = re.sub("판결문|지급명령|이행권고|화해권고", "집행권원", new_name)

        elif re.search("타채|결정문", new_name) :
            new_name = re.sub("타채|결정문", "강제집행", new_name)

        elif re.search("채권양도통지서", new_name) :
            new_name = re.sub("채권양도통지서", "양도통지서", new_name)
        
        elif re.search("[_]개회", new_name) :
            new_name = re.sub("_개회", "_개인회생", new_name)

        elif re.search("[_]신복", new_name) :
            new_name = re.sub("_신복", "_신용회복", new_name)

        if re.search("[_]+보증인", new_name) :
            split_list = re.sub("[_]+보증인", "", new_name).split("_")
            split_list[1] = "보증인" + " "+ split_list[1]
            new_name = "_".join(split_list)

        


        if new_name != f : 
                                        ########
            ff.re_name(join(path,f), join(path, new_name))
                                        ########
    

  0%|          | 0/24184 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

20427510 지성숙_보증인 양도통지서_(1).pdf      20427510_보증인 지성숙_양도통지서_(1).pdf
20427737 봉상준_보증인 양도통지서_(1).pdf      20427737_보증인 봉상준_양도통지서_(1).pdf
20427783 박광식_보증인 양도통지서_(1).pdf      20427783_보증인 박광식_양도통지서_(1).pdf
20427784 김정희_보증인 등본_(1).pdf      20427784_보증인 김정희_등본_(1).pdf
20427784 김정희_보증인 양도통지서_(1).pdf      20427784_보증인 김정희_양도통지서_(1).pdf
20427784 김정희_보증인 초본_(1).pdf      20427784_보증인 김정희_초본_(1).pdf
20427785 이순자_보증인 양도통지서_(1).pdf      20427785_보증인 이순자_양도통지서_(1).pdf
20427786 이은희_보증인 양도통지서_(1).pdf      20427786_보증인 이은희_양도통지서_(1).pdf
20427792 정주희_보증인 양도통지서_(1).pdf      20427792_보증인 정주희_양도통지서_(1).pdf
20427792 정주희_보증인 초본_(1).pdf      20427792_보증인 정주희_초본_(1).pdf
20427816 최현숙_보증인 양도통지서_(1).pdf      20427816_보증인 최현숙_양도통지서_(1).pdf
20427857 송기동_보증인 양도통지서_(1).pdf      20427857_보증인 송기동_양도통지서_(1).pdf
20427857 송기동_보증인 초본_(1).pdf      20427857_보증인 송기동_초본_(1).pdf
20427862 박선화_보증인 양도통지서_(1).pdf      20427862_보증인 박선화_양도통지서_(1).pdf
20427862 박선화_보증인 초본_(1).pdf      20427862_보증인 박선화_초본_(1).pdf
20427908 홍지연_보증인 양도통지서_(1

In [12]:
f = "20427857 송기동_보증인 초본_(1).pdf"


<re.Match object; span=(2, 3), match='기'>


##### 사건번호 다시 넣기

In [ ]:
v2_dict = ff.fileInfoDict(r"D:\부산 정리_v2")

In [ ]:
len(v2_dict) #{채무자키 : {size : fullpath of files in 부산정리_v2}}

In [ ]:
# 사건번호 있는 파일 목록 작성
p_key = re.compile(r"([\d]{8})[\D]") #match
p_event = re.compile(r"[\D]([\d]{4}[ㄱ-ㅎ가-힣]{1,3}[0-9]+)")
p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif|xps)$', re.I)

no_key_error = []
no_file_error = []
result = [] #4740개 나와야


path = r'D:\1.다운로드\__부산 공유폴더_원본'
filelist = os.walk(path)

In [ ]:
index = 0
for root, __dirs__, files in tqdm(filelist) :
    
    for f in files :
        index += 1
        if (p_extension.search(f) != None) : #스캔본인 경우
            
            # [주의] 사건번호에 띄어쓰기 없어야 하므로 파일명에서 띄어쓰기 제거
            nospace_f = re.sub(" ", "", f)

            if p_event.search(nospace_f) != None : # 사건번호 있으면
                if p_key.search(f) != None : # 원본이라 채무자키가 젤 앞에 없는 경우도 많다.
                    
                    key = str(p_key.search(f).group(1)) # str() 유의
                    size = str(os.path.getsize(join(root, f))) # str() 유의
                    event = p_event.search(nospace_f).group(1) #nospace_f, group(1) 유의
                    
                    try :
                        src = v2_dict[key][size] #변경할 파일 풀경로
                        
                        src_dir = os.path.split(src)[0]
                        src_filename = os.path.split(src)[1]
                        src_stem = os.path.splitext(src_filename)[0]
                        src_ext = os.path.splitext(src_filename)[1]
                        
                        # 사건번호 넣기
                        src_split_list = src_stem.split("_")
                        src_split_list.insert(3, event) 
                        
                        dst_filename = "_".join(src_split_list) + src_ext
                        dst = join(src_dir, dst_filename)
                        
                        result.append(ff.re_name(src, dst))

                    except :
                        no_file_error.append([key, size, event, join(root, f)])
                        continue #아래 코드 실행 안 됨
                else :
                    no_key_error.append(join(root, f))
            else : pass
        else : pass

print(index)

In [ ]:
print(result)

##### 양통 차수, 집행권원 재도

In [ ]:
path = r'D:\부산 정리_v2'
filelist = os.walk(path)
result = []

p_재도 = re.compile("_재도$")
p_차수 = re.compile("_([0-9]차)$")

In [ ]:
for root, __dirs__, files in filelist :

    for f in files :

        src = join(root, f)
        stem = os.path.splitext(f)[0]
        ext = os.path.splitext(f)[1]

        if p_재도.search(stem) :
            
            stem = p_재도.sub("", stem)
            split_list = stem.split("_")
            split_list[2]= split_list[2]+" "+"재도"

            dst_filename = "_".join(split_list) + ext
            dst = join(root, dst_filename)
            result.append(ff.re_name(src, dst))
        
        elif p_차수.search(stem) :

            차수 = p_차수.search(stem).group(1)
            stem = p_차수.sub("", stem)
            split_list = stem.split("_")
            split_list[2] = split_list[2]+ " " + 차수

            dst_filename = "_".join(split_list) + ext
            dst = join(root, dst_filename)
            result.append(ff.re_name(src, dst))

print(len(result), "개의 이름 변경")

##### 재산조회, 녹취록 (수작업 함)

### 체크하기

##### >>> 채무자키만

In [ ]:
path = r'D:\부산 정리'
os.chdir(path)
file_list = os.walk(path)

p_key = re.compile(r'[\d]{8}[\D]')
p_notkey = re.compile('[0-9]{9,}')
error = []

for r, d, files in tqdm(file_list) :
    for f in files :
        if f != "Thumbs.db":
            if p_key.match(f) == None :
                error.append(f)
print(error)

##### >>>문서종류만

In [ ]:
p_docu = re.compile('원인서류|채권양도통지서|판결문|지급명령|이행권고|화해권고|타채|결정문|등본|초본|개회|신복|파산|외국인증명')

path = r'D:\부산 정리'
file_list = os.walk(path)
error_short = []
error_docu = []

for r, d, files in tqdm(file_list) :
    for f in files :
        if f != "Thumbs.db":
            splitname = f.split("_")
            if len(splitname)<3 :
                error_short.append(f)
            elif p_docu.search(splitname[2]) == None :
                error_docu.append(f)
print("short :", *error_short, sep="\n")
print("docu :", *error_docu, sep="\n")

### 돌발상황 처리

##### 이름 중간에 들어간 _


In [ ]:
### 등,초본이 초본을 포함해버린다. 이거 수정해야
path = r'D:\부산 정리'

p_docu = re.compile(
    '원인서류|채권양도통지서|판결문|지급명령|이행권고|화해권고|타채|결정문|등본|초본|개회|신복|파산|외국인증명')
p_ = re.compile("_")

os.chdir(path)
file_list = os.walk(path)
fixed = []
count = 0

for r, d, files in tqdm(file_list):
    for f in files:
        if f != "Thumbs.db":
            new_name = ""
            if p_docu.search(f):
                st = p_docu.search(f).start() - 1
                new_name = f[:9] + p_.sub(" ", f[9:st]) + f[st:]  # 앞뒤 언더바 말고 중간에 언더바

                if f != new_name:
                    os.rename(r+"/"+f, r+"/"+new_name)
                    count += 1
                    fixed.append([f, new_name])
                    

print(count, "개의 파일 이름 변경 완료")
print(*fixed, sep="\n")

##### 잘린 외국인 이름 정정

In [ ]:
# 파일명에 보증인 키워드가 있는지만 보고 그거에 맞춰서 ... 

# 채무자조회파일 dict로 불러오기
dict = ff.debtorInfoDict()

path = r'D:\부산 정리'
file_list = os.walk(path)

p_eng = re.compile("[a-zA-Z]+")
p_etc = re.compile("보증인")

count = 0

for root, dir, files in tqdm(file_list) : 
    for f in files : 

        if f != "Thumbs.db":
            new_name = ""
            
            spl_name = f.split("_", maxsplit=2)

            key = spl_name[0]
            debtor_name = spl_name[1]
            others = spl_name[2]

            if p_eng.search(debtor_name) : # 이름에 영어 있는 경우
                if p_etc.search(others) : # 보증인 키워드 있는 경우
                    new_name = key +"_" + str(dict[key].보증인성명)+"_"+ others
                else : # 보증인 키워드 없는 경우
                    new_name = key +"_" + str(dict[key].성명)+"_"+others
                    
                if new_name != f :
                    src = join(root, f)
                    dst = join(root, new_name)
                    ff.re_name(src, dst)
                    count += 1
print(count, "개의 파일명 변경 완료")
                  

### 수정파일 재작업

##### (날짜4~8자리) -> 6자리

In [ ]:
PATH = r'\\192.168.0.75\솔림헬프'
PATH_LOG_SUCCESS = r'\\192.168.0.75\스캔파일\스캔파일 log\success'
PATH_LOG_FAIL = r'\\192.168.0.75\스캔파일\스캔파일 log\fail'


# PATH = r'C:\Users\SL\Desktop\test'
# PATH_LOG_SUCCESS = r'C:\Users\SL\Desktop\test\log\success'
# PATH_LOG_FAIL = r'C:\Users\SL\Desktop\test\log\fail'



file_list = os.walk(PATH)
log = []
error = []
p_day211 = re.compile(r'[\s_]?\((\d\d)(\d)(\d)\)')
p_day212 = re.compile(r'[\s_]?\((\d\d)(\d)(\d\d)\)')
p_day411 = re.compile(r'[\s_]?\(20(2\d)(\d)(\d)\)')
p_day412 = re.compile(r'[\s_]?\(20(2\d)(\d)(\d\d)\)')

for root, __dirs__, files in tqdm(file_list) :
    try :
        for f in files :
            n = f
            if p_day211.search(f) :
                n = p_day211.sub('_\g<1>0\g<2>0\g<3>', n)
                #print("211", f, n)
            elif p_day212.search(f) :
                n = p_day212.sub('_\g<1>0\g<2>\g<3>', n)
                #print("212", f, n)
            elif p_day411.search(f) :
                n = p_day411.sub('_\g<1>0\g<2>0\g<3>', n)
                #print("411", f, n)
            elif p_day412.search(f) :
                n = p_day412.sub('_\g<1>0\g<2>\g<3>', n)
                #print("412", f, n)
            else :pass

            if n != f :
                log.append(ff.re_name(join(root,f), join(root,n)))


    except Exception as e:
        print("===================================")
        print(datetime.today().strftime("%H:%M:%S"))
        print(f, e.__class__, e, sep=" : ")
        print(traceback.format_exc())
        error.append([f, e.__class__, e])
        continue # 반복문 계속 돌아

ff.write_log_csv(log, PATH_LOG_SUCCESS)
ff.write_log_csv(error, PATH_LOG_FAIL)



##### 양통 차수 잘린거 복구

In [4]:
df = pd.read_csv(r"\\192.168.0.75\스캔파일\스캔파일log\success\20220921 130138_9059.csv", header=None)
df.head()

,0,1,2,3
0,/volume1/스캔파일/새 스캔파일(업로드)/나우리,20407905 조경미양도통지서.pdf,20407905_조경미_양도통지서.pdf,/volume1/솔림헬프/2.양도통지서/한울가람/20407905
1,/volume1/스캔파일/새 스캔파일(업로드)/나우리,20407941 강성준 양도통지서 20191008.pdf,20407941_강성준_양도통지서_191008.pdf,/volume1/솔림헬프/2.양도통지서/한울가람/20407941
2,/volume1/스캔파일/새 스캔파일(업로드)/나우리,20407937 전태수-헬프양도통지서.pdf,20407937_전태수 헬프_양도통지서.pdf,/volume1/솔림헬프/2.양도통지서/한울가람/20407937
3,/volume1/스캔파일/새 스캔파일(업로드)/나우리,20407941 강성준양도통지서이력 20191008.pdf,20407941_강성준_양도통지서_이력_191008.pdf,/volume1/솔림헬프/2.양도통지서/한울가람/20407941
4,/volume1/스캔파일/새 스캔파일(업로드)/나우리,20407959 박은혜(78) 순차적양도통지서.pdf,20407959_박은혜(78) 순차적_양도통지서.pdf,/volume1/솔림헬프/2.양도통지서/한울가람/20407959


In [11]:
import os
import pandas as pd
from os.path import join

df = pd.read_csv("/volume1/스캔파일/스캔파일log/success/20220921 130138_9059.csv", header=None)

dst_d = '/volume1/스캔파일/새 스캔파일(업로드)/나우리'

for i in tqdm(range(len(df.index))) :
    try :
        src_d = df.iloc[i,3]
        src_f = df.iloc[i,2]
        dst_f = df.iloc[i,1]
        os.rename(join(src_d, src_f), join(dst_d, dst_f))
    except :
        pass
    
print(i, "개 파일이동")

    
    

  0%|          | 6/9059 [00:00<00:01, 6017.65it/s]

0
1
2
3
4
5


### 채무자키로 보증인 이름 넣기

In [5]:
path = r'D:\key\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\18.(9017) 메리츠캐피탈_솔림헬프\3.채권양도통지서\2차\스캔_보증인'
for f in tqdm(os.listdir(path)) :
    key = f[:8]
    grt = dict_refer[key][3]
    new_name = key + " " + grt + " " + f[8:]
    try :
        os.rename(join(path, f), join(path, new_name))
    except :
        print(f)
        pass

100%|██████████| 174/174 [00:05<00:00, 29.92it/s]


### 상위 폴더로 파일 끄집어내기

In [6]:
path = r'D:\key\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\25.퀀텀_솔림헬프\4.매입전양도통지서\매입전양도통지서'
for root, __dir__, files in tqdm(os.walk(path)) :
    for f in files :
        try :
            os.rename(join(root, f), join(path, f))
        except :
            print(f)
            pass

493it [00:07, 63.32it/s]
